In [2]:
import pandas as pd
import os
import glob
import re

In [ ]:
# read all the csv files from results/final_presentation folder and concatenate them into one dataframe
# then save the dataframe to a csv file
# 1. get the list of csv fileanmes and filepaths
# 2. read the csv files into dataframes
# 3. concatenate the dataframes
# 4. save the concatenated dataframe to a csv file

list_of_files = glob.glob('experiment_artifacts/results/final_presentation_v2/*.csv')
list_of_files.sort()
loss = True
# S_MobileNetV2_1_0_T_resnet32x4_dataset_cifar100_distill_similarity_weight_cls_0.0_use_labels_True_weight_div_0.0_weight_other_1.0_learning_rate_0.001_lr_decay_epochs_150_180_210_trial_0
if loss:
    model_name_template = (
        "S_{}_T_{}_dataset_{}_distill_{}_weight_cls_{}_use_labels_{}_weight_div_{}_weight_other_{}_learning_rate_{}_lr_decay_epochs_{}_loss_{}_trial_{}.csv"
    )
    pattern = model_name_template.format(
        "(.+)", "(.+)", "(.+)", "(.+)", "(.+)", "(.+)", "(.+)", "(.+)", "(.+)", "(.+)", "(.+)", "(.+)"
    )
else:
    model_name_template = (
    "S_{}_T_{}_dataset_{}_distill_{}_weight_cls_{}_use_labels_{}_weight_div_{}_weight_other_{}_learning_rate_{}_lr_decay_epochs_{}_trial_{}.csv"
)

    # Create a regex pattern from the template
    pattern = model_name_template.format(
    "(.+)", "(.+)", "(.+)", "(.+)", "(.+)", "(.+)", "(.+)", "(.+)", "(.+)", "(.+)", "(.+)", "(.+)"
)
# read the csv files into dataframes
dfs = []
folder_list = []
for filename in list_of_files:
    df = pd.read_csv(filename)
    filename_short = filename.split("/")[-1]
    # Match the pattern against the filename
    match = re.match(pattern, filename_short)
    if match:
    # Extract the hyperparameters from the filename
        model_info = match.groups()
        df["student_model"] = model_info[0]
        df["teacher_model"] = model_info[1]
        df["dataset"] = model_info[2]
        df["distill_method"] = model_info[3]
        df["weight_cls"] = model_info[4]
        df["use_labels"] = model_info[5]
        df["weight_div"] = model_info[6]
        df["weight_other"] = model_info[7]
        df["learning_rate"] = model_info[8]
        df["lr_decay_epochs"] = model_info[9]
        if loss:
            df["loss_function"] = model_info[10]
            df["trial_number"] = model_info[11]
        else:
            df["trial_number"] = model_info[10]
        # get the last epoch of the dataframe
        df_good = df[df['test_acc'] > 60] 
        if df_good.shape[0] > 0:
                # replace .csv with empty string
                filename = filename.replace('.csv', '')
                # split the filename by /
                filename = filename.split('/')
                # get the last element of the list
                filename = filename[-1]
                folder_list.append(filename)
    dfs.append(df)

# concatenate the dataframes
df = pd.concat(dfs, ignore_index=True)
# file the concatenated dataframe where test_acc is greater than 60%
df = df[df['test_acc'] > 60] 
# filer for epoch 240
df = df[df['epoch'] == 240]
df.to_csv('experiment_artifacts/results/final_presentation_concatenated_results_v2.csv', index=False)

In [3]:
# move the tensorboard logs to a new folder if the folder name is in the list of files
for folder in folder_list:
    os.system(f"mv /home/hlaing/new/kd-srp/experiment_artifacts/saved_models_final_presentation_v2/students/tensorboard/{folder} /home/hlaing/new/kd-srp/experiment_artifacts/results/tensorboard/")
    # print(f"mv {folder} /home/hlaing/new/kd-srp/experiment_artifacts/results/tensorboard/")

NameError: name 'folder_list' is not defined

In [ ]:
# read the concatenated dataframe
df = pd.read_csv('experiment_artifacts/results/final_presentation_concatenated_results_v2.csv')
# df = df[df['loss'] == "KL"]
# group by the column: loss, use_label and get the mean of the columns
df_grouped = df.groupby(['distill', 'loss', 'use_labels'])
# loop through the groups and print the mean of the columns
for name, group in df_grouped:
    print(name)
    print("mean", group["test_acc"].mean())
    # get the std of the test_acc
    print("std",group["test_acc"].std())
    

('simkd', 'KL', False)
mean 64.015
std 4.447701653663383
('simkd', 'l2', False)
mean 67.78
std 0.07071067811865074
('simkd_mp', 'KL', False)
mean 69.26
std 1.3892084076912279
('simkd_mp', 'KL', True)
mean 66.87666666666667
std 3.7812343663588663
('simkd_mp', 'l2', False)
mean 71.62
std 0.16970562748476778
('simkd_mp', 'l2', True)
mean 72.07
std 0.19798989873222406
('unb_proj', 'KL', True)
mean 72.6
std 0.4101219330881964
('unb_proj', 'l2', True)
mean 71.91499999999999
std 0.14849242404917057
